# Set dataset parameters
Choose the attributes to load the dataset.

In [2]:
detectors = ['mtcnn']  # 'dlib', 'opencv',
embedding_models = ['ArcFace']  # "Dlib", 'VGG-Face','FaceVACs', 'Facenet512'
# TODO:add quality models to experiments
# quality_models = ['ser_fiq', 'tface']


# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [ ]:
calibrators = ['logit', 'KDE']  # 'KDE', , 'isotonic'
calibration_db = ["LFW", "SCFace"]
filters = ['pose']  # todo: change to image filters.

face_image_filters = []  # 'confusion_score','quality'

n_calibration_pairs = 100

## Validation

In [ ]:
years = [2011, 2012]

## Other parameters
Rest of the configuration

In [ ]:
metrics = "cosine"
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [ ]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [ ]:
home = os.path.expanduser("~")

input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset'

output_dir = os.path.join(home, 'video_resources', 'exp_output')


# Preprocess data
Connect to sql database.

In [ ]:
from sql_face.alchemy import get_session
from sql_face.tables import *

In [ ]:
session = get_session(output_dir, db_name)

# Start experiments

In [ ]:
# experimental_setup = ExperimentalSetup(detectors, 
#                                         embedding_models, 
#                                         calibrators, 
#                                         calibration_db, 
#                                         enfsi_years,
#                                         filters, 
#                                         face_image_filters, 
#                                         metrics, 
#                                         n_calibration_pairs,
#                                         embedding_model_as_scorer,
#                                         session)